In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm, chi2
import sys

In [2]:
def DivisiveNormalization(theta, data):
    denom = theta[0] + np.multiply(theta[1], np.linalg.norm(data, theta[2], 1))
    v=np.divide(data.T, denom)
    
    return v

In [3]:
def calcPiProbitQuad(Mi, v):
    
    MiT=np.transpose(Mi, axes=(0,2,1))
    T=v.shape[0]
    [x, w] = np.polynomial.hermite.hermgauss(100)

    #I honestly don't really know how tensordot works, but these lines of code return the correct values
    c = np.tensordot(MiT,v, axes=([1,0]))
    cT=np.transpose(c, axes=(0,2,1))
    vi = cT.diagonal() #This matches vi in MATLAB for s=1, trials 8,10,14
    
    #first part of equation in ProbaChoice.m, line 242
    z1=np.multiply(-2**0.5, vi)

    #second part of equation in ProbaChoice.m, line 242
    z2=np.multiply(-2**0.5, x)

    #These values have been validated
    zz = [z1-ele for ele in z2]

    aa=np.prod(norm.cdf(zz), axis=1)
    #Pi have been validated
    Pi=np.divide(np.sum(np.multiply(w.reshape(100,1), aa), axis=0), np.pi**0.5)
    
    return Pi
    

In [14]:
def calcPiChosen(v, choices):
    
    """v is values from DivisiveNormalization, choices is an array of containing the indices of the chosen options"""

    probs = np.empty((v.shape[1], v.shape[0]))#reverse shape from the data
    #get the size of the choice array. Choice arrays must be the same size
    Jm=v.shape[0]
    temp = np.identity(Jm-1)
    M = np.empty((Jm, Jm-1, Jm))


    for i in range(Jm):
        M[i] = np.concatenate((temp[:,0:i], -1*np.ones((Jm-1,1)), temp[:, i:]), axis=1)

    Mi=M[choices]
    pi = calcPiProbitQuad(Mi, v)
    
    return pi

In [19]:
def calcPiAll(v):
    
    probs = np.empty((v.shape[1], v.shape[0]))#reverse shape from the data
    #get the size of the choice array. Choice arrays must be the same size
    Jm=v.shape[0]
    temp = np.identity(Jm-1)
    M = np.empty((Jm, Jm-1, Jm))


    for i in range(Jm):
        M[i] = np.concatenate((temp[:,0:i], -1*np.ones((Jm-1,1)), temp[:, i:]), axis=1)
    
    for i in range(Jm):
        y=np.array([i]*v.shape[1])

        
        #Matrices for only the chosen options
        Mi=M[y]
        
        pi=calcPiProbitQuad(Mi,v)
        probs[:,i]=pi.T

    return probs

In [24]:
def choose_item(v):
    probs=calcPiAll(v)
    num_subj = v.shape[1]
    Jm = v.shape[0]


    cov = np.ones((Jm, Jm)) * 0.5
    cov[np.arange(Jm), np.arange(Jm)] = 1
    mean = np.zeros(Jm)
    #for i in range(num_it):
    eps = np.random.multivariate_normal(mean, cov, size=num_subj).T
    #print(eps)
    u = v + eps
    item_chosen = u.argmax(axis=0)
    
    return item_chosen


In [9]:
thetaDN=[0.114, 0.177, 1]
v=DivisiveNormalization(theta=thetaDN, data=choice_set_vals)
all_pi = calcPiAll(v=v)
chosen_pi = calcPiChosen(v=v, choices=chosen_vals)
ic = choose_item(v)

(20, 121)
(121, 20)


In [4]:
#Load data from Bollen et al., 2010
choice = pd.read_csv('/Users/amywinecoff/Documents/CITP/Research/Github/AgentChoiceSim/co1_wide.csv')  

#for now, remove the conditions with 5 options so I can figure out the code for a fixed set size
choice = choice[~choice['condition'].isin(['Top5', 'Top5_NR'])]

score_cols = [c for c in choice.columns if 'score' in c]
movie_cols = [c for c in choice.columns if 'movie' in c]
choice_set_vals = np.array(choice[score_cols]/10)

choice['chosen_num']=None
for idx, m in enumerate(movie_cols):
    choice['chosen_num'] = np.where(choice[m]==choice["choice"], idx, choice['chosen_num'])
chosen_vals = np.array(choice['chosen_num'].astype(int).values)

chosen = choice_set_vals[np.arange(len(choice_set_vals)), chosen_vals]


# Simulation of user choices
We simulate 100,000 trials of each of the 3 choice sets and use the values yielded by the `DivisiveNormalization` method + a random noise vector and check that the choice probabilities are roughly in line with the analytic probabilities from `calcPiProbitQuad`.

In [ ]:
#v.shape


In [ ]:
# v = DivisiveNormalization(theta=thetaDN, data=choice_set_vals)
# ic, u = chose_item(theta=thetaDN, data=choice_set_vals, return_utility=True)
# pi=calcPiChosen(theta=thetaDN, data=choice_set_vals, choices=ic)
# print(np.argmax(v, axis=0))
# print(np.argmax(u, axis=0))
# print(sum(np.log(pi)))

In [ ]:
# #v = DivisiveNormalization(theta=thetaDNNull, data=choice_set_vals)
# #ic, u = chose_item(theta=thetaDNNull, data=choice_set_vals, return_utility=True)
# pi=calcPiChosen(theta=thetaDNNull, data=choice_set_vals, choices=ic)
# #print(np.argmax(v, axis=0))
# #print(np.argmax(u, axis=0))
# print(sum(np.log(pi)))

In [ ]:
# #def chose_item_dn(d, num_it=1000, theta = [0.0000, 0.2376, 0.9739])
# theta=thetaDN
# d = np.array([
#              [4, 2.33, 1.875, 1.8, 1.5, 1.495, 1.335, 1.275, 1.125, 1.09, 1, 0.925],              
#              [2.125, 2.125, 2.025, 2.0, 1.875, 1.495, 1.485, 1.335, 1.275, 1.075, 1.0, 0.625],
#              [4.0, 2.17,  2.0, 2.0, 1.875, 1.875, 1.5, 1.485, 1.335, 1.275, 1.09, 1.075],
#             ])
# freq_chosen = np.array([0., 0., 0.])
# num_it = 100000
# v = DivisiveNormalization(theta=theta, data=d)
# # the following covariance matrix has the structure
# # [ 1    0.5    ...    0.5 ]
# # [ 0.5    1    ...    0.5 ]
# # [ 0.5   ...    1    0.5  ]
# # [ 0.5   0.5   ...    1   ]
# cov = np.ones((12, 12)) * 0.5
# cov[np.arange(12), np.arange(12)] = 1
# mean = np.zeros(12)
# for i in range(num_it):
#     eps = np.random.multivariate_normal(mean, cov, size=3).T
#     u = v + eps
#     item_chosen = (u.argmax(axis=0) == (y-1)).astype(float)
#     freq_chosen += item_chosen / num_it
    
# print(freq_chosen)

###Steps to Computing a Power Analysis Given an Experimental Design and value of theta
1. Read in scores into correct np array format
2. Chose the item given its normalized value 
3. Calculate the probability of the chosen item (based on u rather than strict probabilities for option values?)

In [31]:
def calcModelLL(data, theta, **kwargs):
    """Calculates the log likelikihood given theta values for a DN model. If a null model is being tested,
    it will chose the item based on the alternative model, then calculate the probability of that choice, and the 
    log-likelihood given both the alternative model and the null model
    """
    #This is not really right. Need to figure out how to solve the probability issue since this is calculating based on the theoretical prob, which is not the same as the observeed prob
    ##TODO: Fix this so that it works on variable data size. Right now only running on 20-movie decisions
    #probably need to calculate this based on the calculated u, not on the theoretical probs
    v=DivisiveNormalization(theta=theta, data=data)
    item_chosen = choose_item(v)
    #all_pi = calcPiAll(v=v)
    eps = sys.float_info.epsilon  
    #add epsilon to all values to prevent divide by zero error
    chosen_probs = calcPiChosen(v=v, choices=item_chosen) + eps  
    LL = sum(np.log(chosen_probs))
    
    
    null_theta = kwargs.get("null_theta", None)
    if null_theta:
        v_null=DivisiveNormalization(theta=null_theta, data=data)
        null_chosen_probs = calcPiChosen(v=v_null, choices=item_chosen) + eps
        nullLL = sum(np.log(null_chosen_probs))
            
        return LL, nullLL
    
    return LL
    

In [34]:
def MCPowerSimulation(data, alt_theta, null_theta, dof, iterations=100, alpha=0.05):
    
    simulation_stats = []
    
    for i in range(iterations):
        LL, nullLL = calcModelLL(data=choice_set_vals, theta=thetaDN, null_theta=thetaDNNull)
        
        LR = 2*(LL-nullLL)
        #consider using chi2.sf since sometimes it is more accurate? https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html
        p=1 - chi2.cdf(LR, dof)
        
        simulation_stats.append([i, LL, nullLL, LR, p])
    
    simulation_df = pd.DataFrame(simulation_stats,columns = ["iter","altLL", "nullLL", "LR", "p"])
    
    sig_iters = simulation_df[simulation_df["p"]< alpha]
    
    power = sig_iters.shape[0] / simulation_df.shape[0]
    
    return power, simulation_df

In [35]:
thetaDN=[0.114, 0.177, 1]#Webb 2020 sigma and omega only
thetaDNNull = [0.114, 0, 1]#Fix omega to 0 to test hypothesis that normalization occurrs
p, df = MCPowerSimulation(data=choice_set_vals, alt_theta=thetaDN, null_theta=thetaDNNull, dof=1)

In [37]:
df.head()

,iter,altLL,nullLL,LR,p
0,0,-362.565710,-1539.430482,2353.729544,0.0
1,1,-362.333300,-1584.768863,2444.871126,0.0
2,2,-362.925603,-1626.286806,2526.722407,0.0
3,3,-362.680097,-1614.733057,2504.105920,0.0
4,4,-362.802304,-1686.589566,2647.574524,0.0


In [ ]:
def nestedLRT(LL, nullLL):
    
    df = len([ele for idx, ele in enumerate(thetaDN) if thetaDNNull[idx]!=ele])
    LR = 2*(LL-nullLL)
    #consider using chi2.sf since sometimes it is more accurate? https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html
    p=1 - chi2.cdf(LR, df)
  
    return LR, p

In [ ]:
dnLL_preds = calc

In [ ]:
thetaDN=[0.114, 0.177, 1]#Webb 2020 sigma and omega only
thetaDNNull = [0.114, 0, 1]#Fix omega to 0 to test hypothesis that normalization occurrs
dnLL, nullLL, = calcModelLL(data=choice_set_vals, theta=thetaDN, null_theta=thetaDNNull)
print("dnLL = {}, nullLL= {}".format(dnLL, nullLL))
LR, p = nestedLRT(dnLL, nullLL)

print(LR, p)

In [ ]:
thetaDNb=[0.012, 0.412, 25.74]
dnb_probs=calcPiAll(theta=thetaDNb, data=d)

In [ ]:
print(LL, nullLL)
LR = 2*(LL-nullLL)
print(LR)
p=1 - chi2.cdf(LR, 2)
print(p)

In [ ]:
#d = np.array([
 #            [4, 2.33, 1.875, 1.8, 1.5, 1.495, 1.335, 1.275, 1.125, 1.09, 1, 0.925],              
  #           [2.125, 2.125, 2.025, 2.0, 1.875, 1.495, 1.485, 1.335, 1.275, 1.075, 1.0, 0.625],
   #          [4.0, 2.17,  2.0, 2.0, 1.875, 1.875, 1.5, 1.485, 1.335, 1.275, 1.09, 1.075],
    #        ])
#omega allowed to vary. Set to value in Webb et al., 2020
theta_h1 = [1.0, 0.117, 1.0]
#This is the null model that tests that omega != 0
theta_h0 = [theta_h1[0], 0, theta_h1[2]]

LLs = calcModelLL(theta=theta_h1, data=d, null_theta=theta_h0)
#LL_h1 = calcModelLL(theta=theta_h1, data=d)

print("LL for H0 model: {}".format(LLs[1]))
print("LL for H1 model: {}".format(LLs[0]))
#print(LL_h1)#-362.68216377703664
LR = 2*(LLs[0]-LLs[1])
print(LR)

In [ ]:
p=1 - chi2.cdf(LR, 1)
print(p)

In [ ]:
d = choice_set_vals.values
d

In [ ]:
d = choice_set_vals.values /20
#omega allowed to vary. Set to value in Webb et al., 2020
theta_h1 = [0.44, 0.0006, 1.0]
#This is the null model that tests that omega != 0
theta_h0 = [theta_h1[0], 0, theta_h1[2]]

LLs = calcModelLL(theta=theta_h1, data=d, null_theta=theta_h0)
#LL_h1 = calcModelLL(theta=theta_h1, data=d)

print("LL for H0 model: {}".format(LLs[1]))
print("LL for H1 model: {}".format(LLs[0]))
#print(LL_h1)#-362.68216377703664
LR = 2*(LLs[0]-LLs[1])
print(LR)
p=1 - chi2.cdf(LR, 1)
print(p)

In [ ]:
null_probs=calcPiAll(theta=theta_h0, data=d)

In [ ]:
null_probs_df= pd.DataFrame(null_probs)
null_probs_df.head(10)

In [ ]:
choice_set_vals.head(10)

In [ ]:
# save as Python
#!jupyter nbconvert --to script DivisiveNormalization.ipynb

In [ ]:
# data=choice_set_vals
# choices = chosen_vals
# v=DivisiveNormalization(theta=thetaDN, data=choice_set_vals)
# probs = np.empty(data.shape)
# #get the size of the choice array. Choice arrays must be the same size
# Jm=data.shape[1]
# temp = np.identity(Jm-1)
# M = np.empty((Jm, Jm-1, Jm))




# for i in range(Jm):
#     M[i] = np.concatenate((temp[:,0:i], -1*np.ones((Jm-1,1)), temp[:, i:]), axis=1)

# Mi=M[chosen_vals]
# pi = calcPiProbitQuad(Mi, v)

In [ ]:

d = choice_set_vals.values
#sigma, omega(w), beta
#theta_h1 = [0.0000, 0.2376, 0.9739]
probs=calcPiAll(theta=t, data=d)
print(probs)

In [ ]:
def vuong_test(p1, p2):
    r"""
    https://gist.github.com/jseabold/6617976
    Vuong-test for non-nested models.
    Parameters
    ----------
    p1 : array-like
        f1(Y=y_i | x_i)
    p2 : array-like
        f2(Y=y_i | x_i)
    Notes
    -----
    This is hard-coded for testing Poisson vs. Zero-inflated. E.g.,
    it does not account for
    Let f_j(y_i|x_i) denote the predicted probability that random variable Y
    equals y_i under the assumption that the distribution is f_j(y_i|x_i) for
    j = 1,2. Let
    .. math::
       m_i = log(\frac{f_1(y_i|x_i)}{f_2(y_i|x_i)})
    The test statistic from Vuong to test the hypothesis of Model 1 vs.
    Model 2 is
    .. math::
       v = \frac{\sqrt{n}(1/n \sum_{i=1}^{n}m_i)}{\sqrt{1/n \sum_{i=1}^{n}(m_i - \bar{m})^2}}
    This statistic has a limiting standard normal distribution. Values of
    v greater than ~2, indicate that model 1 is preferred. Values of V
    less than ~-2 indicate the model 2 is preferred. Values of |V| < ~2 are
    inconclusive.
    References
    ----------
    Greene, W. Econometric Analysis.
    Vuong, Q.H. 1989 "Likelihood ratio tests for model selection and
        non-nested hypotheses." Econometrica. 57: 307-333.
    """
    m = np.log(p1) - np.log(p2)
    n = len(m)
    v = n ** .5 * m.mean() / m.std()
    return v, stats.norm.sf(np.abs(v))